In [ ]:
import os
from dataclasses import dataclass
from pathlib import Path

In [26]:
prev_dir=os.getcwd()

In [27]:
os.chdir("../")

In [28]:
%pwd

'c:\\Users\\Muhammad Hasnain\\Desktop\\KidneyDiseaseClassification'

In [61]:
@dataclass(frozen=True)
class BaseModelPrepConfigEntity:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    # augmentation: bool = True
    params_include_top: bool
    params_epochs: int
    params_classes: int
    params_batch_size: int
    params_img_shape: list 
    params_learning_rate: float 
    params_weights: list

In [45]:
from cnn_classifier.constants import  *
from cnn_classifier.utils.common import read_yaml,create_directories

In [62]:
class BaseModelPrepConfigManager():
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.params=read_yaml(params_filepath)
        self.config=read_yaml(config_filepath)

        # create_directories([self.config.artifacts_root])
    
    def get_model_preparation_config(self)->BaseModelPrepConfigEntity:
        config=self.config.prepare_base_model
        create_directories([config.root_dir])

        model_preparation_config=BaseModelPrepConfigEntity(root_dir=config.root_dir,base_model_path=Path(config.base_model_path),updated_base_model_path=Path(config.updated_base_model_path),params_include_top=self.params.INCLUDE_TOP,params_classes=self.params.CLASSES,   params_batch_size= self.params.BATCH_SIZE,
         params_img_shape=self.params.IMAGE_SHAPE ,params_learning_rate= self.params.LEARNING_RATE, params_weights=self.params.WEIGHTS,params_epochs=self.params.EPOCHS)

        return model_preparation_config


In [63]:
obj=BaseModelPrepConfigManager()
obj.get_model_preparation_config()

[2025-09-29 00:26:45,897] : common: INFO: yaml file 'params.yaml' loaded 
[2025-09-29 00:26:45,903] : common: INFO: yaml file 'config\config.yaml' loaded 
[2025-09-29 00:26:45,908] : common: INFO: created directory at: artifacts/prepare_base_model


BaseModelPrepConfigEntity(root_dir='artifacts/prepare_base_model', base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/updated_base_model.h5'), params_include_top=False, params_epochs=1, params_classes=2, params_batch_size=16, params_img_shape=BoxList([224, 224, 3]), params_learning_rate=0.01, params_weights='imagenet')

In [64]:
import os
# from urllib.request import requests
import zipfile
from cnn_classifier.utils.common import save_bin
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model


In [82]:
class PrepareBaseModel:
    def __init__(self,config:BaseModelPrepConfigManager):
        self.config=config

    def get_base_model(self):
        model=VGG16(
            input_shape=self.config.params_img_shape,include_top=self.config.params_include_top,
            weights=self.config.params_weights
        )
        self.save_model(path=self.config.base_model_path,model=model)
        self.model=model

    @staticmethod
    def _prepare_full_model(model:Sequential, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        full_model =Sequential([
                model,                         # your pre-trained VGG16 backbone
                Flatten(),                     # flatten the output
                Dense(classes, activation="softmax")  # final prediction layer
            ])
        full_model.compile(
        optimizer=SGD(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"])
        full_model.summary()
        full_model.summary()
        return full_model
    


    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path,model:Sequential):
                 model.save(path)

In [83]:
try:
    model_prep_config=BaseModelPrepConfigManager()
    cnf=model_prep_config.get_model_preparation_config()
    model=PrepareBaseModel(config=cnf)
    model.get_base_model()
    model.update_base_model()
except Exception as e:
    raise e

[2025-09-29 00:40:48,179] : common: INFO: yaml file 'params.yaml' loaded 
[2025-09-29 00:40:48,187] : common: INFO: yaml file 'config\config.yaml' loaded 
[2025-09-29 00:40:48,187] : common: INFO: created directory at: artifacts/prepare_base_model
[2025-09-29 00:40:48,802] : saving_api: WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2025-09-29 00:40:53,720] : saving_api: WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
